In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import gc
import configparser
from datetime import datetime
import time

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import fbeta_score, precision_score, recall_score

from utils.f2thresholdfinder import *
from utils.loaderjpg import *
from utils.generator import *
from utils.custommetrics import *
from utils.visualization import *
from utils.predictorjpg import *
from utils.file import *

from pretrained.vgg16 import *
from pretrained.vgg19 import *
from pretrained.resnet50 import *
from pretrained.densenet121 import *
from pretrained.custommodels import *

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
config_file = 'cfg/default.cfg'

print('reading configurations from config file: {}'.format(config_file))

settings = configparser.ConfigParser()
settings.read(config_file)
data_dir = settings.get('data', 'data_dir')

rescaled_dim = 224

# model_name = 'vgg16'
model_name = 'vgg19'
# model_name = 'resnet50'
# model_name = 'densenet121'

# train top layers only
top_layer_index_dict = {
    'vgg16': 19,
    'vgg19': 38,
    'resnet50': 175,
    'densenet121': 607
}

train_top_classifer = True

file_uuid = time.strftime("%Y%m%d-%H%M%S")

verbose_level = 0

labels = ['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']

reading configurations from config file: cfg/default.cfg


C:\Users\Me\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


In [8]:
frozen_layers = top_layer_index_dict[model_name]

In [3]:
model = custom_top_model(model_name, num_classes=17, num_frozen_layers=frozen_layers)

#print(model.summary())
# check trainability of all layers
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable if hasattr(layer, 'trainable') else False)

(0, 'zeropadding2d_1', False)
(1, 'convolution2d_1', False)
(2, 'zeropadding2d_2', False)
(3, 'convolution2d_2', False)
(4, 'maxpooling2d_1', False)
(5, 'zeropadding2d_3', False)
(6, 'convolution2d_3', False)
(7, 'zeropadding2d_4', False)
(8, 'convolution2d_4', False)
(9, 'maxpooling2d_2', False)
(10, 'zeropadding2d_5', False)
(11, 'convolution2d_5', False)
(12, 'zeropadding2d_6', False)
(13, 'convolution2d_6', False)
(14, 'zeropadding2d_7', False)
(15, 'convolution2d_7', False)
(16, 'zeropadding2d_8', False)
(17, 'convolution2d_8', False)
(18, 'maxpooling2d_3', False)
(19, 'zeropadding2d_9', False)
(20, 'convolution2d_9', False)
(21, 'zeropadding2d_10', False)
(22, 'convolution2d_10', False)
(23, 'zeropadding2d_11', False)
(24, 'convolution2d_11', False)
(25, 'zeropadding2d_12', False)
(26, 'convolution2d_12', False)
(27, 'maxpooling2d_4', False)
(28, 'zeropadding2d_13', False)
(29, 'convolution2d_13', False)
(30, 'zeropadding2d_14', False)
(31, 'convolution2d_14', False)
(32, 'zeropa

In [4]:
df_train = pd.read_csv(data_dir + 'train_v2.csv')
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

(40479L, 224L, 224L, 3L)
(40479L, 17L)


In [5]:
number_of_samples = x_train.shape[0]
split = int(number_of_samples * 0.90)
                     
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

number_validations = number_of_samples - split

In [6]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_2[0][0]            
___________________________________________________________________________________________

In [9]:
# early stopping prevents overfitting on training data
early_stop = EarlyStopping(monitor='val_loss',patience=1, min_delta=0, verbose=0, mode='auto')

model_filepath = data_dir + 'bottleneck/{}/frozen{}_{}.h5'.format(model_name, frozen_layers, file_uuid)
# save only the best model, not the latest epoch model.
model_checkpoint = ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

# also save the weights because Keras raises error in load_model when model contains custom layers
weights_filepath = data_dir + 'bottleneck/{}/frozen{}_{}_weights_only.h5'.format(model_name, frozen_layers, file_uuid)
weights_checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

In [14]:
batch_size = 64

# models with more trainable params require a smaller batch size to fit into the GPU memory of my 1060.
if model_name == 'densenet121':
    batch_size = 32
elif model_name == 'vgg19':
    batch_size = 16

img_normalization = image_normalization_func(model_name)

train_datagen = BottleNeckImgGenerator(normalization=img_normalization)
train_gen = train_datagen.trainGen(x_train, y_train, batch_size)
valid_datagen = BottleNeckImgGenerator(normalization=img_normalization)
valid_gen = valid_datagen.validationGen(x_valid, y_valid, batch_size)

In [15]:
history = {}
f2_history = []

learning_rate_schedule = [0.0001, 0.00005]  # starting at 0.001 does not yield good val_loss. 
max_epoch_per_learning_rate = [70, 10]

# using Adam for top layer.
if model_name == 'densenet121':
    learning_rate_schedule = [0.001, 0.0002]
    max_epoch_per_learning_rate = [60, 10]

if model_name == 'vgg19':
    learning_rate_schedule = [0.01, 0.001]
    max_epoch_per_learning_rate = [10, 10]

if train_top_classifer:
    training_start_time = datetime.now()

    num_samples_per_epoch = x_train.shape[0]
    
    for learn_rate, epochs in zip(learning_rate_schedule, max_epoch_per_learning_rate):
        print('learning rate :{}'.format(learn_rate))
        model.optimizer.lr.set_value(learn_rate)
        
        tmp_history = model.fit_generator(train_gen,
                        samples_per_epoch=num_samples_per_epoch,
                        nb_epoch=epochs,
                        validation_data=valid_gen,
                        nb_val_samples=number_validations,              
                        verbose=verbose_level,
                        callbacks=[early_stop, weights_checkpoint]) # model_checkpoint
    
        for k, v in tmp_history.history.iteritems():
            history.setdefault(k, []).extend(v)

    time_spent_trianing = datetime.now() - training_start_time
    print('top classifier layers training complete. Time taken: {}'.format(time_spent_trianing))


learning rate :0.0001


GpuArrayException: out of memory
Apply node that caused the error: GpuAllocEmpty{dtype='float32', context_name=None}(Assert{msg='The convolution would produce an invalid shape (dim[0] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[1] < 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[2] <= 0).'}.0, Assert{msg='The convolution would produce an invalid shape (dim[3] <= 0).'}.0)
Toposort index: 299
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(16L, dtype=int64), array(64L, dtype=int64), array(224L, dtype=int64), array(224L, dtype=int64)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty{dtype='float32', context_name=None}.0, GpuDnnConvDesc{border_mode='valid', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Co

In [ ]:
# #TODO put this in a utils script
# def normalize_images(images):
#     # int8 to float16, subtract mean, transpose
#     x_result = images.astype(np.float16)
#     subtract_mean(x_result)
#     x_result = x_result.transpose(0,3,1,2) # theano expects channels come before dims
#     return x_result

In [ ]:
if train_top_classifer:
    valid_datagen = BottleNeckImgGenerator(normalization=img_normalization)
    valid_gen = valid_datagen.validationGen(x_valid, y_valid, batch_size)
    
    p_valid = model.predict_generator(valid_gen, x_valid.shape[0])
    y_predictions = (np.array(p_valid) > 0.2).astype(int)

    precision_s = precision_score(y_valid, y_predictions, average='samples')
    print('>>>> Overall precision score over validation set ' , precision_s)

    recall_s = recall_score(y_valid, y_predictions, average='samples')
    print('>>>> Overall recall score over validation set ' , recall_s)

    f2_score = fbeta_score(y_valid, y_predictions, beta=2, average='samples')
    print('>>>> Overall F2 score over validation set ' , f2_score)

In [ ]:
figures_dir = 'figures/{}'.format(model_name)
makedirs(figures_dir)

plot_file_path = figures_dir + '/stats_frozen{}_{}.png'.format(frozen_layers, file_uuid)
trainHistoryPlot(plot_file_path, history, [], None)